# City Bike Initial Data Analysis (IDA)

In this step we are going to dive into the dataset of citibikes riderships of February, 2016 and do an initial data analysis in which we will

* Explore the available variables we have for each observation
* Format the data columns to a correct types as well as check the data sanity of the date columns
* Explore the presence of missing data, how much, in which column and whether there is some pattern in it.

Finally we are going to join with a weather dataset for the same time period and save our processed dataset in a new csv ready for further analysis.

In [1]:
# loading used packages
import pandas as pd
from utilities.date_utilities import is_bike_time_correct, format_bike_times
from utilities.preprocess import attach_additional_bike_columns, cast_bike_types

from dotenv import find_dotenv, load_dotenv
import os

# load .env environment variables
load_dotenv(find_dotenv())

# env constants
RIDES_YEAR = os.environ.get('RIDES_YEAR')
RIDES_MONTH = os.environ.get('RIDES_MONTH')
RIDES_NEXT_MONTH = os.environ.get('RIDES_NEXT_MONTH')
START_TIME_COL = os.environ.get('START_TIME_COL')
STOP_TIME_COL = os.environ.get('STOP_TIME_COL')

# constants
CITIBIKE_FILENAME = "../data/raw/201602-citibike-tripdata.csv"
WEATHER_FILENAME = "../data/raw/weather_data_nyc_centralpark_2016.csv"

# a filename of the dataset after preprocessing
WEATHERLY_BIKES_FILENAME = "../data/processed/weatherly_bikes_2016_02.csv"

TIME_COLUMNS = ['starttime', 'stoptime']

### Loading and preparing the bike ridership data

In [2]:
# loading the bikes dataset
bikes_df = pd.read_csv(CITIBIKE_FILENAME)

####  Format columns, cast to correct types and add new helper ones. Dates sanity check

In [3]:
# checking the sanity of the dates of the bike riderships
starttime_correct = is_bike_time_correct(bikes_df['starttime']).mean() * 100
stoptime_correct = is_bike_time_correct(bikes_df['stoptime']).mean() * 100

print("The starttime column is correct in {p:.2f}% of the cases.".format(p=starttime_correct))
print("The stoptime column is correct in {p:.2f}% of the cases.".format(p=stoptime_correct))

The starttime column is correct in 100.00% of the cases.
The stoptime column is correct in 100.00% of the cases.


In [4]:
# cast start and stop time to correct formats
bikes_df[TIME_COLUMNS] = format_bike_times(bikes_df[TIME_COLUMNS])

In [5]:
# cast to correct types
bikes_df = cast_bike_types(bikes_df)

# add additional helper columns to help with the analysis
bikes_df = attach_additional_bike_columns(bikes_df)

# see the processed dataframe
bikes_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,gender,month,day,start_hour,start_minute,stop_hour,stop_minute,age,tripduration_minutes,tripduration_hours
0,461,2016-02-01 00:00:08,2016-02-01 00:07:49,480,W 53 St & 10 Ave,40.766697,-73.990617,524,W 43 St & 6 Ave,40.755273,...,male,2,1,0,0,0,7,50,7.683333,0.128056
1,297,2016-02-01 00:00:56,2016-02-01 00:05:53,463,9 Ave & W 16 St,40.742065,-74.004432,380,W 4 St & 7 Ave S,40.734011,...,male,2,1,0,0,0,5,39,4.950000,0.082500
2,280,2016-02-01 00:01:00,2016-02-01 00:05:40,3134,3 Ave & E 62 St,40.763126,-73.965269,3141,1 Ave & E 68 St,40.765005,...,male,2,1,0,1,0,5,29,4.666667,0.077778
3,662,2016-02-01 00:01:00,2016-02-01 00:12:02,537,Lexington Ave & E 24 St,40.740259,-73.984092,428,E 3 St & 1 Ave,40.724677,...,female,2,1,0,1,0,12,33,11.033333,0.183889
4,355,2016-02-01 00:01:41,2016-02-01 00:07:36,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,521,8 Ave & W 31 St,40.750967,...,male,2,1,0,1,0,7,38,5.916667,0.098611


#### Percentage of NAN values in each column

In [6]:
bikes_nan_props_ = bikes_df.isna().mean() * 100

# percentage of nan values in each column
bikes_nan_props_[bikes_nan_props_ > 0].map('{:,.2f}%'.format)

birth year    5.32%
age           5.32%
dtype: object

Only birth year has missing values. That is of slightly above 5%. Lets see how missing age is distributed according to missing gender.

In [7]:
# cross tabulation with gender NA values looking for some pattern in missing data
pd.crosstab(bikes_df['gender'], bikes_df['birth year'].isna())

birth year,False,True
gender,,
female,112587,0
male,417215,1
unknown,1246,29825


We can actually see that most people hiding their age are actually hiding their gender as well. Maybe we can find some pattern with the different user types (Customer and Annual Subscriber)?

In [8]:
pd.crosstab(bikes_df['usertype'], bikes_df['birth year'].isna())

birth year,False,True
usertype,,
Customer,0,29825
Subscriber,531048,1


We have only one subscriber whose age is not available. The rest of the ones with unavailable age are customers. One assumption for that might be that this information is not collected in any way for customer types.

##### Exploring Birth year counts

In [9]:
pd.concat(
    [bikes_df['birth year'].value_counts(normalize=False),
     bikes_df['birth year'].value_counts(normalize=True)],
    axis=1,
    keys=('counts','percentage')
).sort_index(ascending=False)

,counts,percentage
2000,23,0.000043
1999,512,0.000964
1998,1363,0.002567
1997,1229,0.002314
1996,2971,0.005595
...,...,...
1907,4,0.000008
1901,2,0.000004
1900,101,0.00019
1899,38,0.000072


We can see that there are 94 people of age 131 and others of age above 100.
This is probably due to erroneous data.

##### Trip duration

In [10]:
print("Trips above 500 hours: ", (bikes_df['tripduration_hours'] > 500).sum())
print("Trips above 72 hours: ", (bikes_df['tripduration_hours'] > 72).sum())
print("----------------------------")
print("99th emperical quantile is {q:.2f}".format(q=bikes_df['tripduration_minutes'].quantile(.99)))

Trips above 500 hours:  2
Trips above 72 hours:  50
----------------------------
99th emperical quantile is 51.79


Lets also do another cross tabulation to see whether longer rides are not correlated with the user types in some manner.

In [11]:
pd.crosstab(
    bikes_df['tripduration_hours'] > 1,
    bikes_df['usertype'],
    margins=True
)

usertype,Customer,Subscriber,All
tripduration_hours,,,
False,27316,529025,556341
True,2509,2024,4533
All,29825,531049,560874


There are some trips longer than 500 hours and 50 above 72 hours, while the 99th percentile is below an hour.

Aproximately 8% of the customers have rided more than an hour, while the subscribed users biking for more than an hour are only 0.3% (less than percent). <br/>
One assumption for that might be that some riders, especially customers, are forgetting to dock their bike after usage. We should keep in mind that fact, since those observations would greatly affect the mean of the distribution of trip durations.

### Dropping erroneous data

In [12]:
# dropping with age above 90
MAX_AGE = 90

drop_mask = bikes_df['age'] > MAX_AGE
print("We are going to drop {p:.2f}% of our data".format(p=drop_mask.mean() * 100))

bikes_df = bikes_df[(~drop_mask | bikes_df['age'].isna())]


We are going to drop 0.05% of our data


### **Joining our data with the weather data**

In [13]:
weather_df = pd.read_csv(WEATHER_FILENAME)
weather_df.head(5)

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,1-1-2016,42,34,38.0,0.00,0.0,0
1,2-1-2016,40,32,36.0,0.00,0.0,0
2,3-1-2016,45,35,40.0,0.00,0.0,0
3,4-1-2016,36,14,25.0,0.00,0.0,0
4,5-1-2016,29,11,20.0,0.00,0.0,0


Next, we should do some date reformatting and additional column additions to the weather dataset before joining it with the bikes date we already have.

In [14]:
# format the weather date to a correct format and join with our dataset
# TODO: move to a the process module

weather_df['date'] = weather_df['date'].apply(pd.to_datetime, format="%d-%m-%Y")

# add additional columns
weather_df['month'] = weather_df['date'].dt.month
weather_df['year'] = weather_df['date'].dt.year
weather_df['day'] = weather_df['date'].dt.day

# drop weather data which is not in february
feb_mask = (weather_df['month'] == int(RIDES_MONTH)) & (weather_df['year'] == int(RIDES_YEAR))
weather_df = weather_df[feb_mask]
weather_df.head()

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,month,year,day
31,2016-02-01,59,44,51.5,0.01,0.0,2,2,2016,1
32,2016-02-02,50,38,44.0,0.00,0.0,T,2,2016,2
33,2016-02-03,59,42,50.5,0.73,0.0,0,2,2016,3
34,2016-02-04,59,44,51.5,T,0.0,0,2,2016,4
35,2016-02-05,44,31,37.5,0.53,2.5,1,2,2016,5


In [15]:
# writing our processed dataframes to csvs
bikes_df.to_csv("../data/processed/bikes_2016_02.csv")
weather_df.to_csv("../data/processed/weather_2016_02.csv")

# weatherly_bikes_df = pd.merge(left=bikes_df, right=weather_df, left_on='day', right_on='day')
# weatherly_bikes_df['average temperature'].describe()

Finally, we write our joined dataset to a new processed csv so that we use it in an exploratory data analysis.